In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.lines import Line2D
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import numpy as np

In [ ]:
# Load shapfile
#Muncipalities
shapefile_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = gpd.read_file(shapefile_path)

#Highways
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = gpd.read_file(shapefile_path_highways)

In [ ]:
# Filter sg_uf = MT
filtered_highways_MT = highways[highways['sg_uf'] == 'MT']

# Group by vl_br and combine geoetries
def combine_geometries(group):
    return unary_union(group['geometry'])

combined_geometries = filtered_highways_MT.groupby('vl_br').apply(combine_geometries).reset_index(name='geometry')

# combined_geometries being a GeoDataFrame
combined_geometries = gpd.GeoDataFrame(combined_geometries, geometry='geometry')
combined_geometries

In [ ]:
#Load data
file_path = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Table 1.xlsx"
colonization = pd.read_excel(file_path, sheet_name='Planilha1')

# Joining data
merged = pd.merge(municipalities, colonization, left_on='NM_MUN', right_on='City', how='inner')

# FIltering Year is not null
colonization_cities = merged[merged['Year'].notna()]

colonization_cities

In [ ]:
# Ploting
fig, ax = plt.subplots(figsize=(10, 10))

municipalities.plot(ax=ax, color='white', edgecolor='black', linewidth=0.5, label='Municipalities')

colonization_cities.plot(ax=ax, column='NM_MUN', cmap='tab20', marker='o', markersize=50, legend=False)

# Verifyng if 'vl_br' has strings
if combined_geometries['vl_br'].dtype == object:  # if string
    # mapping numerical values
    vl_br_mapping = {value: idx for idx, value in enumerate(combined_geometries['vl_br'].unique())}
    combined_geometries['vl_br_numeric'] = combined_geometries['vl_br'].map(vl_br_mapping)
    column_to_plot = 'vl_br_numeric'
else:
    column_to_plot = 'vl_br'

combined_geometries.plot(ax=ax, column=column_to_plot, cmap='tab20', linewidth=2, legend=False)

vl_br_unique = combined_geometries['vl_br'].unique()
vl_br_colors = plt.cm.tab20(np.linspace(0, 1, len(vl_br_unique)))  

legend_elements_highways = [Line2D([0], [0], color=vl_br_colors[i], lw=2, label=f'BR-{vl_br_unique[i]}') for i in range(len(vl_br_unique))]

colonization_unique = colonization_cities['NM_MUN'].unique()
colonization_colors = plt.cm.tab20(np.linspace(0, 1, len(colonization_unique)))  # Gerar cores para cada valor único
legend_elements_colonization = [Line2D([0], [0], color=colonization_colors[i], marker='o', markersize=10, linestyle='None', label=colonization_unique[i]) for i in range(len(colonization_unique))]

unified_legend_elements = legend_elements_highways + legend_elements_colonization

ax.legend(handles=unified_legend_elements, loc='lower left', bbox_to_anchor=(1, 0), prop={'size': 8})

ax.set_xticks([])
ax.set_yticks([])

ax.set_axis_off()

output_path = r"C:\Users\daves\OneDrive\Pessoal\Acadêmico\Mestrado\Dissertação - Execução\Dissertação\Quadros e gráficos\Graph-Colonization with Highways.png"
plt.savefig(output_path, dpi=1200, bbox_inches='tight')

plt.tight_layout()
plt.show()